In [1]:
!pip install py2neo
!pip install py2neo pandas
!pip install neo4jupyter
!pip install neo4j-driver
!pip install neo4j

zsh:1: /usr/local/bin/pip: bad interpreter: /System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: no such file or directory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.0/177.0 kB 1.8 MB/s eta 0:00:0000:0100:01
zsh:1: /usr/local/bin/pip: bad interpreter: /System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: no such file or directory
zsh:1: /usr/local/bin/pip: bad interpreter: /System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: no such file or directory
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 1.1 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  DEPRECATION: neo4jrestclient is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1

In [2]:
from py2neo import Graph, Node, Relationship
import json

In [3]:
# Neo4J Sandbox https://neo4j.com/sandbox/

In [4]:
# Set up Neo4j graph database
uri = "bolt://44.204.246.117:7687"
user = "neo4j"
password = "neutrons-brake-directives"
graph = Graph(uri, auth=(user, password))

ConnectionUnavailable: Cannot open connection to ConnectionProfile('bolt://44.204.246.117:7687')

In [ ]:
with open('significantAdverseDrugReactions.json', 'r') as f:
    adr_data = json.load(f)

with open('knownDrugsAggregated.json', 'r') as f:
    drug_data = json.load(f)

with open('mechanismOfAction.json', 'r') as f:
    moa_data = json.load(f)

with open('targets.json', 'r') as f:
    target_data = json.load(f)

In [ ]:
# Create nodes for drugs
drugs = {}
for drug in drug_data:
    if drug['drugId'] not in drugs:
        drugs[drug['drugId']] = Node('Drug', id=drug['drugId'])
graph.create(list(drugs.values()))

In [ ]:
pathways = {}
for moa in moa_data:
    for target in moa['targets']['list']:
        if target['element'] in targets:
            for chembl in moa['chemblIds']['list']:
                if chembl['element'] not in pathways:
                    pathways[chembl['element']] = Node('Pathway', id=chembl['element'])
graph.create(list(pathways.values()))

In [ ]:
for moa in moa_data:
    for target in moa['targets']['list']:
        if target['element'] in targets:
            for chembl in moa['chemblIds']['list']:
                if chembl['element'] in pathways:
                    rel = Relationship(targets[target['element']], 'TARGETS', pathways[chembl['element']])
                    graph.create(rel)

In [ ]:
# Function Based

In [ ]:
def create_nodes(tx, data, label):
    for row in data:
        node = dict(row)
        tx.run(f"MERGE (n:{label} $props)", props=node)

In [ ]:
def create_relationships(tx):
    tx.run("""
        MATCH (a:Chembl)-[:TARGETS]->(b:Target)-[:INVOLVES]->(c:ApprovedSymbol)-[:PARTICIPATES]->(d:Pathway)
        MERGE (a)-[:CAUSES]->(e:Event)<-[:INVOLVES]-(b)
        MERGE (a)-[:TARGETS]->(d)
        MERGE (c)-[:INVOLVES]->(d)
    """)

In [ ]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

In [ ]:
with driver.session() as session:
    session.write_transaction(create_nodes, significant_adverse_drug_reactions, "Event")
    session.write_transaction(create_nodes, known_drugs_aggregated, "Chembl")
    session.write_transaction(create_nodes, mechanism_of_action, "Action")
    session.write_transaction(create_nodes, targets, "Target")

In [ ]:
driver.close()

In [ ]:
# Define query to create nodes and relationships
query = '''
MERGE (a:Chemical {chembl_id: $a_id})
MERGE (b:AdverseEvent {event: $b_event})
MERGE (c:Pathway {pathway: $c_pathway})
MERGE (d:GeneSymbol {approved_symbol: $d_symbol})
MERGE (e:Target {target_id: $e_id})
MERGE (a)-[:CAUSES]->(b)
MERGE (a)-[:TARGETS]->(c)
MERGE (d)-[:INVOLVES]->(e)
MERGE (d)-[:PARTICIPATES]->(c)
'''

In [ ]:
# Send data to Neo4j
with driver.session() as session:
    for sadr in sadr_data:
        session.run(query, a_id=sadr["chembl_id"], b_event=sadr["event"], c_pathway=None, d_symbol=None, e_id=None)
    for kda in kda_data:
        session.run(query, a_id=kda["drugId"], b_event=None, c_pathway=None, d_symbol=None, e_id=kda["targetId"])
    for moa in moa_data:
        for chembl_id in moa["chemblIds"]["list"]:
            for target in moa["targets"]["list"]:
                session.run(query, a_id=chembl_id["element"], b_event=None, c_pathway=None, d_symbol=None, e_id=target["element"])
    for target in target_data:
        for pathway in target["pathways"]["list"]:
            session.run(query, a_id=None, b_event=None, c_pathway=pathway["element"]["pathway"], d_symbol=target["approvedSymbol"], e_id=target["id"])